# Extracting zipped parsed page xml files

This nb originally extracted the zip files containing the results of Isaac's parsing that were uploaded to the project [github](https://github.com/Southampton-Digital-Humanities/2023_Catalogue-Heading-Detection). Once we took over maintaining the code we've switched to processing the data locally so the zipping is obsolete. The majority of the code is now about parsing the xmls exported from Transkribus using updated versions of the code that Isaac wrote.

In [1]:
import sys
if '../' not in sys.path:
    sys.path.append('../')
import shutil
import glob
import os
from xml.etree import ElementTree as ET
from tqdm import tqdm
from src.data import xml_extraction as xmle

### Extract zips

For when we were extracting zips of Isaac's data

In [ ]:
ic_data_path = "\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula\\split_data"

zips = glob.glob("\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula\\split_data\\*.zip")

# [shutil.unpack_archive(x, x[:-4], format="zip") for x in zips]

### Rename download jobs

In [ ]:
with open("..\\export_job_rename_dict.txt") as f:  # .txt was a result of massaging the pairings out of Rossitza's .xlsx
    rename_lines = f.readlines()
    rename_split = [x.split(":") for x in rename_lines]
    rename_dict = {x[0]:x[1].strip("\n") for x in rename_split}
    
rename_dict

In [ ]:
# for folder in done_zips:
#     renamed = os.path.join(os.path.dirname(folder), export_dict[os.path.basename(folder)])
#     os.rename(folder, renamed)

## Extracting entries from xmls

In [2]:
# currently just working w vol 3, i = 3
i = 3
two_col_loc = f"..\\data\\raw\\BMC_{i}_2\\*\\*.pxml"
four_col_loc = f"..\\data\\raw\\BMC_{i}_4\\*\\*.pxml"
xmls_2, xmls_4 = xmle.gen_xml_paths(two_col_loc), xmle.gen_xml_paths(four_col_loc)
print(f"{len(xmls_2) + len(xmls_4)} xmls found\n"
      f"2 col ({len(xmls_2):03}): {os.path.dirname(xmls_2[0])}\n4 col ({len(xmls_4):03}): {os.path.dirname(xmls_4[0])}")

197 xmls found
2 col (109): ..\data\raw\BMC_3_2\1255673
4 col (088): ..\data\raw\BMC_3_4\1268122


In [3]:
vol_xml_trees = xmle.gen_xml_trees(xmls_2 + xmls_4)

100%|████████████████████████████████████████████████████████████████████████████████| 197/197 [00:09<00:00, 19.86it/s]


In [4]:
for child in vol_xml_trees["J_2704_aa_30_3_0049_4"]:
    print(child.tag, child.attrib)

{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Metadata {}
{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Page {'imageFilename': '51195634.jpg', 'imageWidth': '4047', 'imageHeight': '5785'}


In [5]:
text_regions = [x for x in vol_xml_trees["J_2704_aa_30_3_0049_4"][1] if len(x) > 2]

In [ ]:
print("\nExtracting catalogue entries from xmls")
lines, xml_track_df = xmle.extract_lines_for_vol(vol_xml_trees)
title_shelfmarks, title_indices, ordered_lines = xmle.find_headings(lines)
entry_df = xmle.extract_catalogue_entries(ordered_lines, title_indices, title_shelfmarks, xml_track_df)

### Combined volumes

In [ ]:
# paired globs
combined_volumes = []
for i in range(1):
    network_loc = "\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula"
    suffix = "column pages Transkribus export"
    combined_volumes.append([os.path.join(network_loc, f"BMC_{i+1} 2 {suffix}"), os.path.join(network_loc, f"BMC_{i+1} 4 {suffix}")])    
    
#     When running locally
#     combined_volumes.append([f'C:\\Users\\HLloyd\\Documents\\BMC_{i+1}_2', f'C:\\Users\\HLloyd\\Documents\\BMC_{i+1}_4'])

In [ ]:
combined_volumes

#### Split text

In [ ]:
for i, v in enumerate(combined_volumes):
    print(f"Combining  {v[0]}")
    txts = glob.glob(os.path.join(v[0], "splittextfiles\\*.txt")) + glob.glob(os.path.join(v[1], "splittextfiles\\*.txt"))
    single_vol_dir = os.path.join(os.path.dirname(v[0]), f"BMC_{i+1}")
    if not os.path.exists(single_vol_dir):
        os.mkdir(single_vol_dir)
    split_fname = os.path.join(single_vol_dir, "combinedsplittext_single_line.txt")
    
    with open(split_fname, "w", encoding="utf-8-sig") as split_out:
        for t in tqdm(txts):
            with open(t, encoding="utf-8-sig") as infile:
                entry = infile.readlines()
                dash_line = "-----------------------------------\n"
                non_english_line = "NON-ENGLISH SECTION LASTING"
                english_only_entry = [x.strip("\n") for x in entry if dash_line not in x and non_english_line not in x]
                split_str = " ".join(english_only_entry)
                split_out.write(split_str + "\n")